In [ ]:
!pip install datasets
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import json

In [ ]:
# Read the cleaned dataset
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/HarryPotter_chatbot/HarryPotter_all_cleaned.csv')
df_clean = df.dropna().reset_index(drop=True)
df_clean.shape

(1541, 2)

In [ ]:
# Filter out only Harry Potter's lines for completion
harry_identifier = "哈利"  # The identifier used for Harry in the dataset
harry_df_corrected = df_clean[df_clean['Character'] == harry_identifier]
harry_df_corrected = harry_df_corrected.reset_index(drop=True)

# Initialize the list for formatted data
finetune_data = []

# Iterate over Harry's lines to create the message sequences
for i, row in harry_df_corrected.iterrows():
    # Find the index of the current Harry line in the cleaned dataframe
    index_in_clean_df = df_clean.index[df_clean['Sentence'] == row['Sentence']].tolist()
    if index_in_clean_df:
        # Check if the current Harry line is not the first line in the dataframe
        index = index_in_clean_df[0]
        if index > 0 and df_clean.at[index - 1, 'Character'] != harry_identifier:
            previous_line = df_clean.iloc[index - 1]
            messages_sequence = [
                {'role': 'system', 'content': '接下來的對話中，你是哈利波特，一位自信又勇敢的魔法師'},
                {'role': 'user', 'content': previous_line['Sentence']},
                {'role': 'assistant', 'content': row['Sentence']}
            ]
            finetune_data.append({'messages': messages_sequence})

# Write the formatted data to a JSONL file
output_finetune_file = '/content/drive/MyDrive/Colab Notebooks/HarryPotter_chatbot/finetune_file.jsonl'
with open(output_finetune_file, 'w', encoding='utf-8') as f:
    for item in finetune_data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')


In [ ]:
# finetuning datasets without the sytem content
# Filter out only Harry Potter's lines for completion
harry_identifier = "哈利"  # The identifier used for Harry in the dataset
harry_df_corrected = df_clean[df_clean['Character'] == harry_identifier]
harry_df_corrected = harry_df_corrected.reset_index(drop=True)

# Initialize the list for formatted data
finetune_data = []

# Iterate over Harry's lines to create the message sequences
for i, row in harry_df_corrected.iterrows():
    # Find the index of the current Harry line in the cleaned dataframe
    index_in_clean_df = df_clean.index[df_clean['Sentence'] == row['Sentence']].tolist()
    if index_in_clean_df:
        # Check if the current Harry line is not the first line in the dataframe
        index = index_in_clean_df[0]
        if index > 0 and df_clean.at[index - 1, 'Character'] != harry_identifier:
            previous_line = df_clean.iloc[index - 1]
            messages_sequence = [
                {'role': 'user', 'content': previous_line['Sentence']},
                {'role': 'assistant', 'content': row['Sentence']}
            ]
            finetune_data.append({'messages': messages_sequence})

# Write the formatted data to a JSONL file
output_finetune_file = '/content/drive/MyDrive/Colab Notebooks/HarryPotter_chatbot/finetune_file2.jsonl'
with open(output_finetune_file, 'w', encoding='utf-8') as f:
    for item in finetune_data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

# another dataset for characters

In [4]:
import pandas as pd

# Assuming the dataset is already loaded into 'data'
# For demonstration, let's assume 'data' is your DataFrame from the CSV file
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/HarryPotter_chatbot/chararcters_cleaned.csv')
data.head()

,姓名,性別,角色,學院,魔杖,血液狀況,技能
0,哈利·波特,男性,學生,葛萊分多,"11"" 冬青鳳凰羽毛",混血,蛇佬腔|黑魔法防禦術 |探索者
1,榮恩·衛斯理,男性,學生|哈利波特最好的朋友,葛萊分多,"12"" 灰獨角獸尾毛",純血統,巫師棋|魁地奇守門員
2,妙麗·格蘭傑,女性,學生|哈利波特最好的朋友,葛萊分多,"10⁄4"" 藤木龍心弦",麻瓜出身,幾乎所有的
3,阿不思·鄧不利多,男性,校長|哈利波特最喜歡的教授,葛萊分多,"15"" 夜騏長老尾毛芯",混血,被許多人認為是他那個時代最強大的巫師之一
4,魯伯·海格,男性,鑰匙與場地的守護者 |神奇動物保護教授,葛萊分多,"16"" 未知橡木芯",半人（半巨人）,


In [8]:
transformed_data_hp_tone = []

system_message = {"role": "system", "content": "接下來的對話中，你是哈利波特，一位自信又勇敢的魔法師"}

for index, row in data.iterrows():
    # Enhanced character description with a "Harry Potter" tone
    character_description_hp_tone = f"在霍格華茲，{row['姓名']}不是一個普通的角色。{row['姓名']}是{row['性別']}魔法師。\
    他是{row['學院']}的，扮演著{row['角色']}的角色，展現出非凡的魔法天賦。他的魔杖，{row['魔杖']}，是他忠實的夥伴，共同經歷了無數的冒險與挑戰。\
    他是一位{row['血液狀況']}的血統。\ {row['姓名']}的技能是{row['技能']}。"

    # Adjust for characters without a death date

    # Adding the system message and the assistant's character description with a "Harry Potter" tone
    transformed_entry_hp_tone = {
        "messages": [
            system_message,
            {"role": "user", "content": "請以魔法世界的風格描述這個角色。"},
            {"role": "assistant", "content": character_description_hp_tone}
        ]
    }

    transformed_data_hp_tone.append(transformed_entry_hp_tone)

# Save the transformed data with "Harry Potter" tone to a new file
transformed_file_path_hp_tone = '/content/drive/MyDrive/Colab Notebooks/HarryPotter_chatbot/finetune_character.jsonl'
with open(transformed_file_path_hp_tone, 'w', encoding='utf-8') as f:
    for entry in transformed_data_hp_tone:
        f.write(f"{entry}\n")